<a href="https://colab.research.google.com/github/fred2167/ResNet-50-FPN/blob/master/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transform
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
from torchsummary import summary

to_float_cuda = {"dtype": torch.float32, "device":"cuda"}

In [ ]:
transformer = transform.Compose([
                                transform.Resize(256),
                                transform.CenterCrop(224),
                                transform.ToTensor(),
                                transform.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = torchvision.datasets.CIFAR10("./data",train=True, transform=transformer,download=True)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2)

dev_dataset = torchvision.datasets.CIFAR10("./data",train=False, transform=transformer,download=True)
devloader = torch.utils.data.DataLoader(dev_dataset,batch_size=64,shuffle=True,num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class SE_block (nn.Module):
  '''
  Squeeze and Excitation block
  Some channels may be more important than another, therefore create a weight for number
  of channels allow the network to prioritize different channels.
  Sigmoid is used to ensure the outputs are between 0 and 1.

  Inputs:
  - in_channel: num of channels for the input feature maps
  - ratio: paper default value, used to reduce dimension in linear layers

  Outpus:
  - A weight layer of shape (C,)
  '''
  def __init__(self,in_channel,ratio=16):
    super().__init__()

    self.block = nn.Sequential(
                                nn.AdaptiveAvgPool2d((1,1)),
                                nn.Flatten(),
                                nn.Linear(in_channel,in_channel // ratio),
                                nn.ReLu(),
                                nn.Linear(in_channel // ratio, in_channel),
                                nn.Sigmoid()
                                )

    def forward (self,x):
      return self.block(x)

class MBConv_block(nn.Module):

  def __init__ (self,in_channel, out_channel=None, kernel_size=3, channel_expand_factor=6,padding="same"):
    super().__init__()

    self.in_channel = in_channel
    self.out_channel = out_channel

    # Not changing channels
    if out_channel is None:
      out_channel = in_channel

    if padding == "same":
      pad = (kernel_size - 1) // 2
    else:
      pad = 0

    self.SE_block = SE_block(in_channel)

    self.inv_bottleneck_block = nn.Sequential(
                                    nn.Conv2d(in_channel,in_channel*channel_expand_factor,kernel_size=1,bias=False,padding=pad),
                                    nn.BatchNorm2d(in_channel*channel_expand_factor),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channel*channel_expand_factor,in_channel*channel_expand_factorpand_factor,kernel_size=kernel_size,bias=False,padding=pad),
                                    nn.BatchNorm2d(in_channel*channel_expand_factor),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channel*channel_expand_factor,in_channel,kernel_size=1,bias=False,padding=pad),
                                    nn.BatchNorm2d(out_channel)
                                    )
  def forward (self,x,debug=False):

    output = self.inv_bottleneck_block(x)
    weight = self.SE_block(output)


    if debug:
      print(f"MBConv output shape: {output.shape}")

    return output * weight +  x


  class MBConv_Stage(nn.Module):

    def __init__(self, in_channel, out_channel, kernel_size, channel_expand_factor, num_base_layer, layer_coefficient, down_sample = False);
    super().__init__()

    self.stage = 



In [ ]:
class efficientNet (nn.Module):

  def __init__(self,fi=0):
    super().__init__()

    alpha = 1.2 **fi    #Depth, num of layers
    beta = 1.1 **fi     #Width, num of channels
    gemma = 1.15 **fi   #Resolution, Only influence stage 1 depth


    stage_1_param = {"output_channels": round(32 * beta),
                     "kernel_size": 3,
                     "num_layers": round(1 * gemma)}

    stage_1 = nn.ModuleList([
                             nn.Conv2d(3,stage_1_param["output_channels"],stage_1_param["kernel_size"]) for i in range(stage_1_param["num_layers"])
                             ]) 
    stage_2_param = {"output_channels": round(16 * beta),
                     "kernel_size": 3,
                     "channel_expand_factor": 1,
                     "num_layers": round(1 * alpha)}

